<a href="https://colab.research.google.com/github/Shiva0krishna/chatbot-finetuned-llm/blob/main/custom_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [6]:

dataset = load_dataset('json', data_files='/content/sample_data/conversations.json')


Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [19]:
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples['input_text'],
        examples['response_text'],
        truncation=True,
        padding='max_length'
    )
    # Add 'decoder_input_ids'
    model_inputs['decoder_input_ids'] = model_inputs['input_ids']
    # Create labels (shift decoder_input_ids and ignore loss for padding)
    labels = model_inputs['decoder_input_ids'].copy()
    labels = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels
    ]
    model_inputs['labels'] = labels
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [15]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer


In [21]:
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['train']
)

In [23]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=7.741078058878581, metrics={'train_runtime': 57.2975, 'train_samples_per_second': 0.105, 'train_steps_per_second': 0.052, 'total_flos': 812050808832.0, 'train_loss': 7.741078058878581, 'epoch': 3.0})

In [24]:
model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-model')


('./fine-tuned-model/tokenizer_config.json',
 './fine-tuned-model/special_tokens_map.json',
 './fine-tuned-model/vocab.txt',
 './fine-tuned-model/added_tokens.json',
 './fine-tuned-model/tokenizer.json')

In [27]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load fine-tuned model
model = AutoModelForSeq2SeqLM.from_pretrained('./fine-tuned-model')
tokenizer = AutoTokenizer.from_pretrained('./fine-tuned-model')

# Function to generate response
def generate_response(input_text):
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding='max_length')
    outputs = model.generate(**inputs)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example usage
input_text = "Hello! How are you?"
response = generate_response(input_text)
print(response)


? [unused11]? you? you?
